In [1]:
import re
import pandas as pd

from fuzzywuzzy import fuzz
from fuzzywuzzy import process

c:\users\karthikeya\appdata\local\programs\python\python37\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
form_adv_firm_names = pd.read_csv('../data/form_adv_formadvfirm.csv')
formd_firm_names = pd.read_csv('../data/formd_firms_firm.csv')

In [3]:
formd_firm_names.head()

,cik_no,crd_no,name
0,1583728.0,167139.0,Altalis Capital Management
1,1578258.0,164338.0,Billings Capital Management
2,NaN,160989.0,Cedar Cove Capital
3,1301396.0,115167.0,Context Capital Management
4,1546283.0,146382.0,Glovista Investments


In [4]:
form_adv_firm_names.head()

,id,crd_no,name
0,62091,324504,HAT TRICK CAPITAL LLC
1,62092,324497,DAYS GLOBAL ADVISORS
2,62062,319586,"LGVP MANAGEMENT, LLC"
3,9749,116913,"ANCHOR BAY CAPITAL, INC."
4,7966,171653,MH ADVISORY SERVICES LLC


In [5]:
formd_firm_names.rename(columns={'name' : 'formd_firms'}, inplace=True)
form_adv_firm_names.rename(columns={'name' : 'formadv_firms'}, inplace=True)

In [6]:
formd_firm_names.drop_duplicates(subset=['formd_firms'], inplace=True)

In [7]:
formd_firm_names = formd_firm_names.iloc[:100]

In [8]:
def prefix_num(val):
    idx = 0
    while idx < len(val) - 1 and not val[idx].isalpha():
        idx += 1

    special_char = {'LLC', 'INC', 'LLP'}
    newVal = []
    for name in val[idx:].split():
        if name in special_char:
            continue
        alterName = re.sub('%d', '', name)
        newVal.append(alterName)

    return ' '.join(newVal)

In [9]:
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace(',', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace('(', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace(')', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace('"', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace('-', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace('&', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.replace('/', '')
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].apply(prefix_num)
form_adv_firm_names['formadv_firms'] = form_adv_firm_names['formadv_firms'].str.title()

In [10]:
form_adv_firm_names

,id,crd_no,formadv_firms
0,62091,324504,Hat Trick Capital Llc
1,62092,324497,Days Global Advisors
2,62062,319586,Lgvp Management Llc
3,9749,116913,Anchor Bay Capital Inc.
4,7966,171653,Mh Advisory Services Llc
...,...,...,...
61906,11284,105519,Barrow Hanley Global Investors
61907,32815,114409,Dick Rivera & Associates
61908,52343,306395,Duro Capital Singapore Advisors Pte. Ltd.
61909,57937,316778,Dkm Wealth Management Inc.


In [11]:
formd_firm_names['formd_firms'] = formd_firm_names['formd_firms'].str.replace(',', '')
formd_firm_names['formd_firms'] = formd_firm_names['formd_firms'].str.replace('(', '')
formd_firm_names['formd_firms'] = formd_firm_names['formd_firms'].str.replace(')', '')

In [12]:
name2match = formd_firm_names['formd_firms'].to_list()
namechoice = form_adv_firm_names['formadv_firms'].to_list()[:1000]

In [13]:
def checker(wrong_options, correct_options):
    names_array=[]
    ratio_array=[]
    for wrong_option in wrong_options:
        if wrong_option in correct_options:
            names_array.append(wrong_option)
            ratio_array.append('100')
        else:
            x = process.extractBests(wrong_option, correct_options, scorer=fuzz.token_set_ratio)
            names_array.append(x[0])
            ratio_array.append(x[1])

    return names_array, ratio_array

In [14]:
name_match, ratio_match = checker(name2match, namechoice)

result_df = pd.DataFrame()

result_df['formd_firms'] = pd.Series(name2match)
result_df['formadv_firms'] = pd.Series(name_match)
result_df['ratio'] = pd.Series(ratio_match)

In [15]:
result_df.head()

,formd_firms,formadv_firms,ratio
0,Altalis Capital Management,Lcv Capital Management,90
1,Billings Capital Management,Lcv Capital Management,90
2,Cedar Cove Capital,Dt Capital,82
3,Context Capital Management,Lcv Capital Management,90
4,Glovista Investments,Dean Investments,81


In [16]:
result_df.to_csv('../output/firm_names_fuzzy_scores.csv')